In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Big Query

In [2]:
from google.cloud import bigquery

In [3]:
client=bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In [ ]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [ ]:
tables = list(client.list_tables(dataset))

In [ ]:
for table in tables:
    print(table.table_id)

In [ ]:
table_ref = dataset_ref.table("full")
table = client.get_table(table_ref)

In [ ]:
table.description

In [ ]:
table.num_rows

## Table Schema

In [ ]:
table.schema

In [ ]:
client.list_rows(table,max_results=5)

In [ ]:
client.list_rows(table,max_results=5).to_dataframe()

In [ ]:
client.list_rows(table, selected_fields=table.schema[:1], max_results = 5).to_dataframe()

## Select, From and Where in Bigquery

In [ ]:
dataset_ref=(client.dataset("openaq", project = "bigquery-public-data"))
dataset=client.get_dataset(dataset_ref)

tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

In [ ]:
table_ref = dataset_ref.table("global_air_quality")
table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

In [ ]:
query="""SELECT city
         FROM `bigquery-public-data.openaq.global_air_quality`
         WHERE country = 'US'"""

In [ ]:
client=bigquery.Client()
query_job = client.query(query)

us_cities = query_job.to_dataframe()

us_cities

In [ ]:
us_cities.city.value_counts().head()

Since there is a limit of 5TB, you can first calculate the size of the query:

In [ ]:
query="""SELECT city,country
         FROM `bigquery-public-data.openaq.global_air_quality`
         WHERE country = 'US'"""

dry_run_config = bigquery.QueryJobConfig(dry_run = True)

dry_run_query_job=client.query(query,job_config = dry_run_config)

print(f'This query will process {dry_run_query_job.total_bytes_processed} bytes')

In [ ]:
print(f'This query will process {(dry_run_query_job.total_bytes_processed)/1000000} mb')

In [ ]:
print(f'This query will process {(dry_run_query_job.total_bytes_processed)/1000000000} gb')

You can also specify a parameter when running the query to limit how much data you are willing to scan. Here's an example with a low limit.

In [ ]:
# Only run the query if it's less than 1 MB
ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)

# Set up the query (will only run if it's less than 1 MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
safe_query_job.to_dataframe()

In [ ]:
# Only run the query if it's less than 1 MB
ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=100000000)

# Set up the query (will only run if it's less than 1 MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
safe_query_job.to_dataframe()

The query gets cancelled if the limit of 1 MB exceeds. However, we can increase the limit to run the query successfully!

In [ ]:
# Only run the query if it's less than 1 GB
ONE_GB = 1000*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

# Set up the query (will only run if it's less than 1 GB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
job_post_scores = safe_query_job.to_dataframe()

# Print average score for job posts
job_post_scores.score.mean()

## GroupBy, Having and Count

In [4]:
dataset_ref = client.dataset("hacker_news", project = "bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("comments")
table = client.get_table(table_ref)

client.list_rows(table, max_results = 5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  import sys


,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


The `parent` column indicates the comment that was replied to and `id` is unique used to identify each comment

In [7]:
query_popular = """SELECT parent, COUNT(id)
                   FROM `bigquery-public-data.hacker_news.comments`
                   GROUP BY parent
                   HAVING COUNT(id)>10"""

In [8]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
query_job = client.query(query_popular, job_config=safe_config)

popular_comments = query_job.to_dataframe()
popular_comments.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,parent,f0_
0,4684384,87
1,6584683,47
2,9616946,78
3,7750036,57
4,8185461,63


If you are ever unsure what to put inside the **COUNT()** function, you can do `COUNT(1)` to count the rows in each group. Most people find it especially readable, because we know it's not focusing on other columns. It also **scans less data** than if supplied column names (making it faster and using less of your data access quota).

In [9]:
query_improved = """SELECT parent, COUNT(1) AS Num_Posts FROM `bigquery-public-data.hacker_news.comments`
                    GROUP BY parent
                    HAVING COUNT(1)>10"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)

query_job = client.query(query_improved,job_config = safe_config)

improved_df = query_job.to_dataframe()
improved_df.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,parent,Num_Posts
0,7536283,45
1,4053076,242
2,2530963,59
3,1934367,70
4,8204007,43


Note that because it tells SQL how to apply aggregate functions (like COUNT()), it doesn't make sense to use GROUP BY without an aggregate function. Similarly, if you have any GROUP BY clause, then all variables must be passed to either a

GROUP BY command, or
an aggregation function.

```
query_good = """
             SELECT parent, COUNT(id)
             FROM `bigquery-public-data.hacker_news.comments`
             GROUP BY parent
             """
```

Note that there are two variables: `parent` and `id`.

parent was passed to a GROUP BY command (in `GROUP BY parent`), and
id was passed to an aggregate function (in `COUNT(id)`).

And this query won't work, because the `author` column isn't passed to an aggregate function or a GROUP BY clause:

```
query_bad = """
            SELECT author, parent, COUNT(id)
            FROM `bigquery-public-data.hacker_news.comments`
            GROUP BY parent
            """
```

***If make this error, you'll get the error message SELECT list expression references column (column's name) which is neither grouped nor aggregated at.***